In [18]:
from nltk import PCFG,ViterbiParser
from nltk import word_tokenize
pcfg=PCFG.fromstring("""
S -> NP VP [1.0]
PP -> P NP [1.0]
VP -> V NP [0.7]
VP -> VP PP [0.3]
P -> 'with' [1.0]
V ->'saw' [1.0]
NP -> NP PP [0.4]
NP -> 'Mani' [0.1]
NP -> 'ears' [0.18]
NP -> 'saw' [0.04]
NP -> 'stars' [0.18]
NP -> 'telescope' [0.1]
"""
)
text="Mani saw stars with ears"
tokens=word_tokenize(text)
n=len(tokens)
print(tokens)
table=[[[]for i in range(n)]for i in range(n)]
print(table)
def addprob(table,left,i,j,prod_prob):
    list1=[prod for prod in pcfg.productions() if prod.rhs()==(left,)]
    if len(list1)==0:
        return ;
    for prod in pcfg.productions():
        if prod.rhs()==(left,):
            prod=prod_prob*prod.prob()
            table[i][j].append([prod.lhs(),prod_prob])
            addprob(table,prod.lhs(),i,j,prob)

for i in range(n):
    for prod in pcfg.productions(rhs=tokens[i]):
        if table[i][i] is None:
            table[i][i]=[(prod.lhs(),prod.prob())]
        else:
            table[i][i].append([prod.lhs(),prod.prob()])
        addprob(table,prod.lhs(),i,i,prod.prob())    
print(table)            
for length in range(2,n+1):
    for i in range(n-length+1):
        j=i+length-1
        for k in range(i,j):
            for prod in pcfg.productions():
                for left,leftprob in table[i][k]:
                    for right,rightprob in table[k+1][j]:
                        if prod.rhs()==(left,right):
                            prob=leftprob*rightprob*prod.prob()
                            if table[i][j] is None:
                                table[i][j]=[(prod.lhs(),prob)]
                            else:
                                table[i][j].append([prod.lhs(),prob])
                            addprob(table,prod.lhs(),i,j,prod.prob())    
print(table)                                
if table[0][n-1] is not None:
    if pcfg.start() in (x for (x,prob) in table[0][n-1]):
        print("Sentence is valid.")
        for (x,prob) in table[0][n-1]:
            print("Probability of sentence is ",prob)  
        print("Max probability sentence is ",max(table[0][n-1]))

['Mani', 'saw', 'stars', 'with', 'ears']
[[[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []]]
[[[[NP, 0.1]], [], [], [], []], [[], [[V, 1.0], [NP, 0.04]], [], [], []], [[], [], [[NP, 0.18]], [], []], [[], [], [], [[P, 1.0]], []], [[], [], [], [], [[NP, 0.18]]]]
[[[[NP, 0.1]], [], [[S, 0.0126]], [], [[S, 0.0009071999999999999], [S, 0.0006804]]], [[], [[V, 1.0], [NP, 0.04]], [[VP, 0.126]], [], [[VP, 0.009071999999999998], [VP, 0.006803999999999999]]], [[], [], [[NP, 0.18]], [], [[NP, 0.01296]]], [[], [], [], [[P, 1.0]], [[PP, 0.18]]], [[], [], [], [], [[NP, 0.18]]]]
Sentence is valid.
Probability of sentence is  0.0009071999999999999
Probability of sentence is  0.0006804
Max probability sentence is  [S, 0.0009071999999999999]


In [19]:
parser=ViterbiParser(pcfg)
for tree in parser.parse(tokens):
    print(tree)

(S
  (NP Mani)
  (VP (V saw) (NP (NP stars) (PP (P with) (NP ears))))) (p=0.0009072)
